## An rag workflow for evaluating the performance of Performed interview.


In [71]:

import json
import os


def load_transcripts(file_path):
    """
    Load transcripts from a JSON file.

    Args:
        file_path (str): The path to the JSON file containing the transcripts.
    
    Returns:
        dict: A dictionary containing the transcripts.
    
    Raises:
        FileNotFoundError: If the file doesn't exist.
        json.JSONDecodeError: If the file contains invalid JSON.
    """
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        
        with open(file_path, 'r', encoding='utf-8') as f:
            transcripts = json.load(f)
            return transcripts
    
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        raise
    except Exception as e:
        print(f"Error loading transcripts: {e}")
        raise

print(load_transcripts(r'../../interviews/627dc248.json'))


{'tech_stack': 'html, css ,js', 'position': 'frontend developer', 'question_count': 5, 'difficulty': 'beginner', 'conversation_history': [{'role': 'interviewer', 'content': "Hello! I'm your AI interviewer for today's frontend developer interview.\n\nI see your tech stack includes: html, css ,js\n\nLet's start with something fundamental. Can you explain what html is and describe one project where you've used it effectively?"}, {'role': 'candidate', 'content': 'HTML (HyperText Markup Language) is the standard language used to structure content on the web, defining elements like headings, paragraphs, links, and images.It forms the backbone of every webpage, often paired with CSS for styling and JavaScript for interactivity.I used HTML effectively in a personal portfolio project, where I built a responsive multi-page site to showcase my development skills.It included semantic HTML tags for better accessibility and SEO.The project integrated images, forms, and navigation menus, giving a pro

In [48]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")


from typing_extensions import TypedDict
from langgraph.graph import StateGraph, END,START



from schema import (
    TechnicalSkillAssessment,
    ProblemSolvingInstance,
    EvaluationWorkFlowState,
    ConfidenceLevel,
    ProficiencyLevel
)


In [88]:
def llm2_problem_solving_skill_evaluator(state:EvaluationWorkFlowState) -> EvaluationWorkFlowState:
    """Problem solving skill Evaluator"""

    print('llm2 got called')

    try:
        system_prompt = """
         You are a senior software developer specialized in evaluating problem solving abilities and implementation abilities.

        your task is to evaluate candidate's approach to a problem, ability to analyze it and quality of implemented solution.
        
        Focus on:
        1. Problem solving mehod and attempted solution
        2. Effectiveness of solution
        3. Ability to analising the problem and logical reasoning
        4. Quality of approach taken by the candidate
        5. Debugging potential and troubleshooting skills.
        6.Creativity in finding solutions

        Return your response in JSON structure following the format given below:
        {
            "problem_solving_instances":[{
                "problem_statement": "E-commerce real-time inventory challenge",
                "solution": "Implemented a live websocket based live inventory tracking system integrated with redis for fast cache updates",
                "approach_quality":9,
                "solution_effectiveness": 8,
                "reasoning_clarity":9
            }],
            "analytical_thinking_score": 7,
            "implementation_ability_score": 6,
            "debugging_potential_score": 7,
            "problem_solving_approach": "Systematic approach with consideration of real-world constraints",
        }
   
        Evaluate specific instances where the candidate solved problems or explained their approach."""

        human_prompt=f"""Analyze this interview conversation for problem-solving and implementation abilities:

Interview Data: {state["interview_data"]}

Return only valid JSON following the specified structure.
        
    """
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=human_prompt)
        ]

        response = llm.invoke(messages)

        print("Raw LLM Response:")
        print(response.content)

        parser = JsonOutputParser()
        result = parser.parse(response.content)

        state['problem_solving_instances'] = result.get("problem_solving_instances", [])
        state['analytical_thinking_score'] = result.get("analytical_thinking_score", 0)
        state['implementation_ability_score'] = result.get("implementation_ability_score", 0)
        state['debugging_potential_score'] = result.get("debugging_potential_score", 0)
        state['problem_solving_approach'] = result.get("problem_solving_approach", "")
        state['current_step'] = "llm2_completed"
        print("LLM2 completed successfully!")

    except Exception as e:
        print(f"Error: {e}")
        raise

    return state

In [89]:
def llm1_technical_evaluator(state: EvaluationWorkFlowState) -> EvaluationWorkFlowState:
    """LLM 1: Technical Skills Evaluator - Updates technical fields in state"""
    print('LLM1 got called') 

    # print(f'State = {state['interview_data']}')
    try:
        system_prompt = """You are a Senior Technical Interviewer specializing in evaluating technical skills and knowledge 
        depth.

Your task is to analyze the interview conversation and assess the candidate's technical competencies.

Focus on:
1. Depth of technical understanding
2. Specific skills demonstrated (HTML, CSS, JavaScript, frameworks, etc.)
3. Quality of technical explanations
4. Knowledge gaps and areas needing improvement

Return your analysis in JSON format with the following structure:
{
    "technical_skills": [
        {
            "skill_name": "JavaScript",
            "proficiency_level": "intermediate",
            "evidence": ["Explained closures correctly", "Mentioned ES6 features"],
            "confidence": "high",
            "comments": "Good understanding shown"
        }
    ],
    "technical_consistency_score": 7,
    "technical_depth_score": 6,
    "technical_knowledge_gaps": ["Advanced React patterns", "Testing frameworks"],
    "technical_strengths": ["Strong JavaScript fundamentals", "Good understanding of async programming"]
}

IMPORTANT: 
- Use "evidence" not "evidance"
- Use "confidence" not "confidence_level" 
- Always include "comments" field for each skill
- proficiency_level must be one of: "beginner", "intermediate", "advanced", "expert"
- confidence must be one of: "low", "medium", "high", "very_high"

Be thorough but fair in your assessment."""

        human_prompt = f"""Analyze this interview conversation for technical skills:

Interview Data: {state['interview_data']}

Return only valid JSON following the specified structure."""

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=human_prompt)
        ]
        print('calling llm1')
        response = llm.invoke(messages)
        print("Raw LLM Response:")
        print(response.content)
        
        # Parse JSON response
        parser = JsonOutputParser()
        result = parser.parse(response.content)
        
        # Update state with technical evaluation results using bracket notation
        state['technical_skills'] = result.get("technical_skills", [])
        state['technical_consistency_score'] = result.get("technical_consistency_score", 0)
        state['technical_depth_score'] = result.get("technical_depth_score", 0)
        state['technical_knowledge_gaps'] = result.get("technical_knowledge_gaps", [])
        state['technical_strengths'] = result.get("technical_strengths", [])
        state['current_step'] = "llm1_completed"
        
        print("LLM1 completed successfully!")
        
    except KeyError as e:
        error_msg = f"LLM1 Key error - missing field: {str(e)}"
        state['errors'].append(error_msg)
        print(f"Error: {error_msg}")
        
    except json.JSONDecodeError as e:
        error_msg = f"LLM1 JSON parsing error: {str(e)}"
        state['errors'].append(error_msg)
        print(f"Error: {error_msg}")
        
    except Exception as e:
        error_msg = f"LLM1 Technical Evaluator error: {str(e)}"
        state['errors'].append(error_msg)
        print(f"Error: {error_msg}")
        
    return state

if __name__ == "__main__":
    # Load interview data
    interview_data2 = load_transcripts(r'../../interviews/627dc248.json')
    print("Loaded interview data:")
    print(interview_data2)
    
    # Create an initial state with required fields
    initial_evaluation_state = {
        "interview_data": interview_data2,
        "current_step": "start",
        "errors": [],  # Initialize empty errors list
        "technical_skills": [],
        "technical_consistency_score": 0,
        "technical_depth_score": 0,
        "technical_knowledge_gaps": [],
        "technical_strengths": []
    }
    
    # Call the technical evaluator function
    print('calling llm with updated state')
    state_after_technical_evaluation_update = llm1_technical_evaluator(initial_evaluation_state)
    state_after_problem_solving_skill_evaluation_update = llm2_problem_solving_skill_evaluator(initial_evaluation_state)

    print("State after technical evaluation update:")


print("Technical Skills:")
for skill in state_after_technical_evaluation_update['technical_skills']:  # Changed from updated_state.technical_skills
    print(f"  - {skill['skill_name']}: {skill['proficiency_level']}")  # Changed to bracket notation
    print(f"    Confidence: {skill['confidence']}")
    print(f"    Evidence: {skill['evidence']}")
    print(f"    Comments: {skill['comments']}")
    print()

print("Technical Consistency Score:", state_after_technical_evaluation_update['technical_consistency_score'])  # Changed to bracket notation
print("Technical Depth Score:", state_after_technical_evaluation_update['technical_depth_score'])  # Changed to bracket notation
print("Knowledge Gaps:", state_after_technical_evaluation_update['technical_knowledge_gaps'])  # Changed to bracket notation
print("Technical Strengths:", state_after_technical_evaluation_update['technical_strengths'])  # Changed to bracket notation
print("Errors:", state_after_technical_evaluation_update['errors'])  # Changed to bracket notation

print('-'*60)
print("State after problem_solving evaluation update:")

for skill in state_after_problem_solving_skill_evaluation_update['problem_solving_instances']:
    print(f"  - {skill['problem_statement']}: {skill['approach_quality']}")  # Changed to bracket notation
    print(f"    Solution: {skill['solution']}")
    print(f"    Confidence: {skill['solution_effectiveness']}")
    print(f"    Evidence: {skill['reasoning_clarity']}")
    print()


print(" problem_solving_score : ",state_after_problem_solving_skill_evaluation_update["problem_solving_score"])
print(" analytical thinking score : ",state_after_problem_solving_skill_evaluation_update['analytical_thinking_score'])
print("debugging_potential_score:",state_after_problem_solving_skill_evaluation_update['debugging_potential_score']  )
print("problem_solving_approach:",state_after_problem_solving_skill_evaluation_update['problem_solving_approach']  )
print("Errors:", state_after_problem_solving_skill_evaluation_update['errors'])  # Changed to bracket notation



Loaded interview data:
{'tech_stack': 'html, css ,js', 'position': 'frontend developer', 'question_count': 7, 'difficulty': 'beginner', 'conversation_history': [{'role': 'interviewer', 'content': "Hello! I'm your AI interviewer for today's frontend developer interview.\n\nI see your tech stack includes: html, css ,js\n\nLet's start with something fundamental. Can you explain what html is and describe one project where you've used it effectively?"}, {'role': 'candidate', 'content': 'HTML (HyperText Markup Language) is the standard language used to structure content on the web, defining elements like headings, paragraphs, links, and images.It forms the backbone of every webpage, often paired with CSS for styling and JavaScript for interactivity.I used HTML effectively in a personal portfolio project, where I built a responsive multi-page site to showcase my development skills.It included semantic HTML tags for better accessibility and SEO.The project integrated images, forms, and navigat

ValueError: Invalid format specifier ' "E-commerce real-time inventory challenge",
                "solution": "Implemented a live websocket based live inventory tracking system integrated with redis for fast cache updates",
                "approach_quality":9,
                "solution_effectiveness": 8,
                "reasoning_clarity":9
            ' for object of type 'str'